In [14]:
#Creating the default Dataframe
import pandas as pd 
import numpy as np 
import pandas_profiling
import datetime
from geopy.geocoders import Nominatim
import re
from coordinates_function import grabCoordinates

#Original file 
xlsx_path = '/Users/chrisrios/Desktop/Data_science/airbnb_post_merge/Airbnb_Project/zillow_data/Zillow_Austin_housing.xlsx'

#Making the Dataframe
zillow_df = pd.read_excel(xlsx_path)

#output the dataframe
print(zillow_df)

      ID                                               File    city stateCode  \
0      0  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1      1  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
2      2  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
3      3  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
4      4  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
...   ..                                                ...     ...       ...   
1226  26  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1227  27  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1228  28  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1229  29  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   
1230  30  <_io.TextIOWrapper name='/Users/bhava/Desktop/...  Austin        TX   

      zipCode              

In [2]:
#rename columns to have no spaces
zillow_df.rename(columns = 
   {
    'Bedroom':'num_bedrooms', 
    'Bathroom':'num_bathrooms', 
    'Sqft':'sqft',
    'Zestimate':'home_price',
    'Address':'street_address',
    'City':'city',
    'Zip Code':'zipcode', 
    'latestPrice':'rent_price',
    'Latest_salemonth':'latest_salemonth'}, inplace = True)



In [3]:
#Data Validation 

#Remove text from bedrooms
zillow_df['num_bedrooms'] =  zillow_df['num_bedrooms'].str.replace('bd', '')

#Getting rid of blank spaces 
try:
    zillow_df['city'] = zillow_df['city'].str.strip()
except: print("City column already deleted")

#Change weird characters to NaNs
zillow_df = zillow_df.replace('--', np.NaN)
zillow_df = zillow_df.replace(' --', np.NaN)

#replacing empty number fields with NaN
zillow_df['rent_price'] = zillow_df['rent_price'].replace(' ', np.NaN)
zillow_df['num_bathrooms'] = zillow_df['num_bathrooms'].replace(' ', np.NaN)
zillow_df['num_bedrooms'] = zillow_df['num_bedrooms'].replace(' ', np.NaN)
zillow_df['home_price'] = zillow_df['home_price'].replace(' ', np.NaN)
zillow_df['sqft'] = zillow_df['sqft'].replace(' ', np.NaN)
zillow_df['rent_price'] = zillow_df['rent_price'].replace(' ', np.NaN)

#Cleaning up rent 
zillow_df['rent_price'] = zillow_df['rent_price'].replace("$","")
zillow_df['rent_price'] = zillow_df['rent_price'].replace(",","")

#Move high rent prices to the home prices (misconfigured originally)
for index,row in zillow_df.iterrows():
      #Grabbing the original string
      if zillow_df.loc[index , 'rent_price'] > 50000:
            rent_string = zillow_df.loc[index , 'rent_price']
            zillow_df.loc[index,'home_price'] = rent_string
            zillow_df.loc[index,'rent_price'] = np.NaN

#Drop NaNs that cant be interpolated
zillow_df.dropna(subset='num_bathrooms',inplace=True)

#Replace missing sqft with NaN
zillow_df['sqft'].fillna(np.nan)

#Drop bad years (1900)
zillow_df.drop(zillow_df.index[zillow_df['latest_saleyear'] == 1900], inplace = True)



#Drop Non Austin Cities 
"""
Drop all cities that arent Austin 
    -everything after drop is the criteria
    -df.index selects 
"""
try:
    zillow_df.drop(zillow_df[zillow_df['city'] != "Austin"].index, inplace=True)
except:
    print("city column already deleted")

#Drop irrelevent columns 
try:
    zillow_df.drop(columns=['city','Acres', 'numPriceChange','State','rent_price'],inplace=True)
except: print("The columns have already been deleted")

In [4]:
# Change repetitive data into Categories (efficiency)
def convertToCategory(*args):
    for col in args:
        zillow_df[col] = zillow_df[col].astype('category')

def convertToFloat(*args):
    for col in args:
        zillow_df[col] = zillow_df[col].astype('float64')

#Convert dates to Datetime DT
def convertToDt(*args):
    for arg in args:
        zillow_df[arg] = pd.to_datetime(zillow_df[arg])

def replaceCharacters(*args, column, newvalue):
    """Replaces x strings of characters, takes column and the new value to replace the values with """
    for oldvalue in args:
        zillow_df[column] = zillow_df[column].replace(oldvalue,newvalue)

convertToFloat('num_bedrooms','num_bathrooms','sqft')


In [5]:
#Round Bathrooms
bathlambda = lambda bath : round(bath * 2) / 2
zillow_df['num_bathrooms'] = bathlambda(zillow_df['num_bathrooms'])

In [6]:
#Data Interpolation

In [8]:
#Create Square footage bins
zillow_df['sqft_bins'] = pd.cut(zillow_df['sqft'], 10)


In [9]:
#Count missing bedrooms
Nanbedrooms = zillow_df['num_bedrooms'].isna().sum()
print("There were " + str(Nanbedrooms) + " missing bedrooms")

#fill Bedrooms based on bathrooms + zipcode
zillow_df['num_bedrooms'] = zillow_df.groupby(['sqft_bins', 'num_bathrooms']).num_bedrooms.bfill()
zillow_df['num_bedrooms'] = zillow_df.groupby(['sqft_bins', 'num_bathrooms']).num_bedrooms.ffill()

Nanbedrooms = zillow_df['num_bedrooms'].isna().sum()
print("After backfilling there are now " + str(Nanbedrooms) + " missing bedrooms")

There were 902 missing bedrooms
After backfilling there are now 9 missing bedrooms


In [10]:
#Drop the remaining missing
zillow_df['num_bedrooms'].dropna(inplace=True)
zillow_df['sqft'].dropna(inplace=True)


In [11]:
#Samuel told me to rather drop this column. IGNORE

    #Count missing rents
    #nanRent = zillow_df['rent_price'].isna().sum()
    #print("There were " + str(nanRent) + " missing rent")

    ##fill Rent based on closest column that shares bathrooms + zipcode + Bedrooms
    #zillow_df['rent_price'] = zillow_df.groupby(['sqft', 'num_bedrooms']).rent_price.bfill()
    #nanRent = zillow_df['rent_price'].isna().sum()
    #print("After backfilling there are now " + str(nanRent) + " missing rent")

In [ ]:
#reorder Dataframe
zillow_df = zillow_df[[
    'street_address', 
    'latitude',
    'longitude',
    'zipcode' , 
    'num_bedrooms', 
    'num_bathrooms', 
    'home_price' , 
    'latest_saledate',
    'latest_salemonth',
    'latest_saleyear',
    'URL'
    ]]

In [ ]:
#Check Data Types
zillow_df.dtypes

street_address       object
zipcode               int64
num_bedrooms        float64
num_bathrooms       float64
home_price          float64
latest_saledate       int64
latest_salemonth      int64
latest_saleyear       int64
URL                  object
dtype: object

In [ ]:
def generate_Report():
    #Generate the profile Report
    profile = pandas_profiling.ProfileReport(zillow_df, title = "Zillow Profiling Report") 
    #Display within the notebook
    profile.to_notebook_iframe()

#generate_Report()

In [47]:
    zillow_df.to_csv('/Users/chrisrios/Desktop/Data_science/Airbnb_Project/zillow_data/cleaned_zillow_sales.csv')